In [ ]:
import warnings
warnings.filterwarnings('ignore')


from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load an image from file


def vgg_16():
    img = "/Users/cgarg/Documents/charu/intuit/code/face/data/lfw/Ed_Wade/Ed_Wade_0001.jpg"
    image = load_img(img, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # load the model
    vggModel = VGG16()
    # predict the probability across all output classes
    yhat = vggModel.predict(image)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # print the classification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

# 1. Train, Test, Valid data split

In [ ]:
import math
from collections import Counter
# Fractions for each dataset
train_frac = 0.70
valid_frac = 0.05
test_frac = 0.25
 
# This function takes in np arrays of images and labels along with split fractions
# and returns the six data arrays corresponding to each dataset as the appropriate type
def create_data_splits(X, y, train_frac=train_frac, test_frac=test_frac, valid_frac=valid_frac):
    X = np.array(X)
    y = np.array(y)
#     print("x shape:: ", X.shape)
#     print("y shape:: ", Y.shape)
    labels = np.array([])
    for index in range(len(y)):
#         print(y[index][0])
        labels = np.append(labels,inv_class_mapping[y[index][0]])
    y = labels
 
 # Make sure that the fractions sum to 1.0
    assert (test_frac + valid_frac + train_frac == 1.0), "Test + Valid + Train Fractions must sum to 1.0"
 
    X_raw_test = []
    X_raw_valid = []
    X_raw_train = []
 
    y_raw_test = []
    y_raw_valid = []
    y_raw_train = []
 
 # Randomly order the data and labels
#     random_indices = np.random.permutation(len(X))
    random_indices = np.random.permutation(len(X))
#     print(random_indices)
#     print("X len ", len(X))
#     print("Y len ",len(Y))
    X = X[random_indices]
    y = y[random_indices]
#     print("class images :", class_images)
 
    for image, label in zip(X, y):
         
 # Number of images that correspond to desired fraction
#         print(label , ":::", class_images[label])
        test_length = math.floor(test_frac * class_images[label])
        valid_length = math.floor(valid_frac * class_images[label])
 
 # Check to see if the datasets have the right number of labels (and images)
        if Counter(y_raw_test)[label] < test_length:
            X_raw_test.append(image)
            y_raw_test.append(label)
        elif Counter(y_raw_valid)[label] < valid_length:
            X_raw_valid.append(image)
            y_raw_valid.append(label)
        else:
            X_raw_train.append(image)
            y_raw_train.append(label)
#     print(type(X_raw_train), ":: ", type(X_raw_valid), ": ", type(X_raw_test), ":: ", type(y_raw_train), ":: ", type(y_raw_valid), ":: ", type(y_raw_test))
    np.array(X_raw_train)
    return np.array(X_raw_train), np.array(X_raw_valid), np.array(X_raw_test), np.array(y_raw_train), np.array(y_raw_valid), np.array(y_raw_test)
# Create all the testing splits using the create_splits function
X_train, X_valid, X_test, y_train, y_valid, y_test = create_data_splits(X_transfer_learning, Y_sample)

In [ ]:
# print(image.shape)

In [ ]:
# from scipy.misc import imresize
from PIL import Image
# Function takes in an image array and returns the resized and normalized array
def prepare_image(image, target_height=299, target_width=299):
    if(type(image) == np.ndarray):
        image = image[0].reshape(250,250,3)
    else:   
        image = image.iloc[0].reshape(250,250,3)
    image = np.array(Image.fromarray(obj=image, mode='RGB').resize(size=(target_height,target_width), resample=Image.BICUBIC))
    return image.astype(np.float32) / 255

In [ ]:
# Function takes in an array of images and labels and processes the images to create
# a batch of a given size
def create_batch(X, y, start_index=0, batch_size=4):
#     print("create batch : start_index:: ", start_index, "batch size:: ", batch_size)
    stop_index = start_index + batch_size
    prepared_images = []
    labels = []
 
    for index in range(start_index, stop_index):
        if(type(X) == np.ndarray):
            preparedImage = prepare_image(X[index]).reshape(299,299, 3)
        else:    
            preparedImage = prepare_image(X.iloc[index]).reshape(299,299, 3)
        dim = np.zeros((299,299))
        #preparedImage = np.stack((preparedImage,preparedImage, preparedImage), axis=-1)
        prepared_images.append(preparedImage)
        
        if(type(y) == np.ndarray):
            labels.append(int(y[index]))
        else:    
            labels.append(inv_class_mapping[y.iloc[index][0]])
 
 # Combine the images into a single array by joining along the 0th axis
    X_batch = np.stack(prepared_images)
    y_batch = np.array(labels, dtype=np.int32)
 
    return X_batch, y_batch

In [ ]:
X_valid, y_valid = create_batch(X_transfer_learning, Y_sample, 0, len(X_transfer_learning))
print("valid data shapes, X_valid: ",X_valid.shape, ", y_valid: ", y_valid.shape)

# 2. Setup Inception ResNet V2

In [ ]:
# example of loading the inception v3 model
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
# load model
model = InceptionResNetV2()
# summarize the model
model.summary()


In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import decode_predictions

def predict_resnet_v2():
    img = "/Users/cgarg/Documents/charu/intuit/code/face/data/lfw/Elaine_Chao/Elaine_Chao_0001.jpg"
    image = load_img(img, target_size=(299, 299))
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    ypred = model.predict(image)
    # convert the probabilities to class labels
    outLabel = decode_predictions(ypred)
    # retrieve the most likely result, e.g. highest probability
    outLabel = outLabel[0][0]
    # print the classification
    print('%s (%.2f%%)' % (outLabel[1], outLabel[2]*100))

In [ ]:
import tensorflow as tf
from tensorflow.contrib import slim
from tensorflow.contrib.slim import nets
from tensorflow.contrib.slim.nets import inception
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from tensorflow.python.tools.optimize_for_inference_lib import optimize_for_inference
# from sklearn.preprocessing import inception_preprocessing


tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 299, 299, 3], name='X')
is_training = tf.placeholder_with_default(False, [])
# Run inception function to determine endpoints

with slim.arg_scope(inception.inception_v3_arg_scope()):
     logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=is_training)
        
inception_saver = tf.train.Saver()
#print(end_points)

In [ ]:
# Isolate the trainable layer
prelogits = tf.squeeze(end_points['PreLogits'], axis=[1,2])
# Define the training layer and the new output layer
n_outputs = len(class_mapping)
with tf.name_scope("new_output_layer"):
    people_logits = tf.layers.dense(prelogits, n_outputs, name="people_logits")
    probability = tf.nn.softmax(people_logits, name='probability')
# Placeholder for labels
y = tf.placeholder(tf.int32, None)
# Loss function and training operation
# The training operation is passed the variables to train which includes only the single layer
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=people_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
 # Single layer to be trained
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="people_logits")
 # The variables to train are passed to the training operation
    training_op = optimizer.minimize(loss, var_list=train_vars)

    # Accuracy for network evaluation
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(predictions=people_logits, targets=y, k=1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
 # Intialization function and saver
with tf.name_scope("init_and_saver"):
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
print("Done setup with n_outputs: ",n_outputs)

In [ ]:
# # Trusty machine learning imports
# import tensorflow as tf
# import numpy as np
# # Make sure to run notebook within slim folder
# from datasets import dataset_utils
# import os
# # Base url
# TF_MODELS_URL = "http://download.tensorflow.org/models/"
# # Modify this path for a different CNN
# INCEPTION_V3_URL = TF_MODELS_URL + "inception_v3_2016_08_28.tar.gz"
# # Directory to save model checkpoints
# MODELS_DIR = "models/cnn"
# INCEPTION_V3_CKPT_PATH = MODELS_DIR + "/inception_v3.ckpt"
# # Make the model directory if it does not exist
# if not tf.gfile.Exists(MODELS_DIR):
#     tf.gfile.MakeDirs(MODELS_DIR)
# # Download the appropriate model if haven't already done so
# if not os.path.exists(INCEPTION_V3_CKPT_PATH): 
#     dataset_utils.download_and_uncompress_tarball(INCEPTION_V3_URL, MODELS_DIR)

In [ ]:
with tf.name_scope("tensorboard_writing"):
 # Track validation accuracy and loss and training accuracy
    valid_acc_summary = tf.summary.scalar(name='valid_acc', tensor=accuracy)
    valid_loss_summary = tf.summary.scalar(name='valid_loss', tensor=loss)
    train_acc_summary = tf.summary.scalar(name='train_acc', tensor=accuracy)
# Merge the validation stats
    valid_merged_summary = tf.summary.merge(inputs=[valid_acc_summary, valid_loss_summary])
# Use the time to differentiate the different training sessions
from datetime import datetime
import time
# Specify the directory for the FileWriter
now = datetime.now().strftime("%Y%m%d_%H%M%S")
model_dir = "{}_unaugmented".format(now)
logdir = "tensorboard/faces/" + model_dir
file_writer = tf.summary.FileWriter(logdir=logdir, graph=tf.get_default_graph())

# 3. Train

In [ ]:
from datetime import datetime
# X_train = X_transfer_learning
# X_valid = X_transfer_learning
# y_train = Y_sample
# y_valid = Y_sample
n_epochs = 100
batch_size = 32
# Early stopping parameters
max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.float("inf")
# Show progress every show_progress epochs
show_progress = 1
# Want to iterate through the entire training set every epoch
n_iterations_per_epoch = len(X_train) // batch_size
# Specify the directory for the FileWriter
now = datetime.now().strftime("%Y%m%d_%H%M%S")
model_dir = "{}_unaugmented".format(now)
logdir = "tensorboard/faces/" + model_dir
file_writer = tf.summary.FileWriter(logdir=logdir, graph=tf.get_default_graph())
# This is the pre-trained model checkpoint training path
inception_v3_checkpoint_path = "models/cnn/inception_v3.ckpt"
# This is the checkpoint path for our trained model with no dataaugmentation
unaugmented_training_path = "models/cnn/inception_v3_faces_unaugmented.ckpt"

with tf.Session() as sess:
    init.run()
 # Restore all the weights from the original CNN
    inception_saver.restore(sess, inception_v3_checkpoint_path)
 
    t0 = time.time()
    for epoch in range(n_epochs):
        start_index = 0
 # Each epoch, iterate through all the training instances
        for iteration in range(n_iterations_per_epoch):
             X_batch, y_batch = create_batch(X_train, y_train, start_index, batch_size)
 # Train the trainable layer 
             sess.run(training_op, {X: X_batch, y: y_batch})
             start_index += batch_size
 
 # Display the progress of training and write to the TensorBoard directory
 # for later visualization of the training
        if epoch % show_progress == 0:
            train_summary = sess.run(train_acc_summary, {X: X_batch, y: y_batch})
            file_writer.add_summary(train_summary, (epoch+1))
 #Size for validation limited by GPU memory (68 images will work)
            valid_loss, valid_acc, valid_summary = sess.run([loss, accuracy, valid_merged_summary], {X: X_valid, y: y_valid})
            file_writer.add_summary(valid_summary, (epoch+1))
            print('Epoch: {:4} Validation Loss: {:.4f} Accuracy: {:4f}'.format(epoch+1, valid_loss, valid_acc))
 
 # Check to see if network is still improving, if improved during epoch
 # a snapshot of the model will be saved to retain the best model
            if valid_loss < best_loss:
                best_loss = valid_loss
                checks_without_progess = 0
                save_path = saver.save(sess, unaugmented_training_path)

 # If network is not improving for a specified number of epochs, stop training
            else:
                checks_without_progress += 1
                if checks_without_progress > max_checks_without_progress:
                    print('Stopping Early! Loss has not improved in {} epochs'.format(max_checks_without_progress))
                    break
 
    t1 = time.time()
 
    print('Total Training Time: {:.2f} minutes'.format( (t1-t0) / 60))

# 4. Compute Test Accuracy 

In [ ]:
eval_batch_size = 32
# X_test = X_transfer_learning
# y_test = Y_sample
n_iterations = len(X_test)//eval_batch_size
with tf.Session() as sess:
 # Restore the new trained model 
 saver.restore(sess, unaugmented_training_path)
 
 start_index = 0
 # Create a dictionary to store all the accuracies
 test_acc = {}
 
 t0 = time.time()
 # Iterate through entire testing set one batch at a time
 for iteration in range(n_iterations):
    X_test_batch, y_test_batch = create_batch(X_test, y_test, start_index, batch_size=eval_batch_size)
    test_acc[iteration] = accuracy.eval({X: X_test_batch, y:y_test_batch})
    start_index += eval_batch_size
    print('Iteration: {} Batch Testing Accuracy: {:.2f}%'.format(
    iteration+1, test_acc[iteration] * 100))
 
    t1 = time.time()
 
 # Final accuracy is mean of each batch accuracy
print('\nFinal Testing Accuracy: {:.4f}% on {} instances.'.format(
np.mean(list(test_acc.values())) * 100, len(X_test)))
print('Total evaluation time: {:.4f} seconds'.format((t1-t0)))